# benjihillard dictionary processing

I found a french dictionary here: https://github.com/Kartmaan/french-language-tools?tab=readme-ov-file

In [129]:
import pandas as pd
from tqdm import tqdm
import ast

In [130]:
raw = pd.read_csv("../../../RawData/Dictionaries/French/kartmann.csv")

In [131]:
raw.sample(4)

,Mot,Définitions
910384,Rerigidifions,"[""Première personne du pluriel de l'indicatif ..."
603257,Ballonnaises,['Féminin pluriel de ballonnais.']
901515,Repromulguez,"[""Deuxième personne du pluriel de l'indicatif ..."
49848,Femtohenrys,['Pluriel de femtohenry.']


## Multiple definitions

All the definitions of a word are collected in the one entry, we'll have to chop them up.


In [132]:
raw[raw.Mot == "Livre"]["Définitions"].tolist()[0]

'[\'Ensemble de pages reliées entre elles et dotées de signes destinés à être lus.\', "Ouvrage de l\'esprit, en prose ou en vers, d\'assez grande étendue pour faire au moins un volume.", \'Écrit sur une face et se présentant sous forme de rouleau.\', \'Ensemble de feuilles de parchemin ou de papier écrites des deux côtés et rassemblées en cahiers, codex.\', \'Ouvrage imprimé, relié ou broché, non périodique, comportant un assez grand nombre de pages.\', "Ce type d\'ouvrage, publié sur un autre support.", \'Sous-partie, division de ces ouvrages.\', \'Terme générique faisant référence à un ouvrage servant de base à une religion et considéré comme sacré.\', "Commerce Registre sur lequel on inscrit ce qu\'on reçoit et ce qu\'on dépense, ce qu\'on achète et ce qu\'on vend, ses dettes actives et ses dettes passives, etc.", "Diplomatie Accompagné d\'une épithète de couleur (désignant la couverture) : Recueil de pièces diplomatiques, publié par le gouvernement d\'un pays.", "Secteur d\'activit

In [133]:
mot = "Livre"
D = raw.loc[raw['Mot'] == mot]['Définitions']
D = eval(D.values[0])

print("Mot : ", mot)
for idx, definition in enumerate(D):
    print(f"{idx+1}) {definition}")

Mot :  Livre
1) Ensemble de pages reliées entre elles et dotées de signes destinés à être lus.
2) Ouvrage de l'esprit, en prose ou en vers, d'assez grande étendue pour faire au moins un volume.
3) Écrit sur une face et se présentant sous forme de rouleau.
4) Ensemble de feuilles de parchemin ou de papier écrites des deux côtés et rassemblées en cahiers, codex.
5) Ouvrage imprimé, relié ou broché, non périodique, comportant un assez grand nombre de pages.
6) Ce type d'ouvrage, publié sur un autre support.
7) Sous-partie, division de ces ouvrages.
8) Terme générique faisant référence à un ouvrage servant de base à une religion et considéré comme sacré.
9) Commerce Registre sur lequel on inscrit ce qu'on reçoit et ce qu'on dépense, ce qu'on achète et ce qu'on vend, ses dettes actives et ses dettes passives, etc.
10) Diplomatie Accompagné d'une épithète de couleur (désignant la couverture) : Recueil de pièces diplomatiques, publié par le gouvernement d'un pays.
11) Secteur d'activité regro

## Capitalisation

Already we see a problem - all words are capitalised, however some words' meaning changes based on capitalisation, such as the constellation vs disease cancer.

We assume the root of a word to be the uncapitalised form, unless it is a proper noun.

## Conclusion
 
We'll lowercase everything and separate the defintions, we accept that dictionaries can be fallable. 

In [134]:
def lowercase_first_letter(s):
    return s[0].lower() + s[1:] if s else s

In [135]:
raw['Mot'] = raw['Mot'].apply(lowercase_first_letter) # lowercase
raw.reset_index(inplace = True)

In [136]:
raw.sample(3)

,index,Mot,Définitions
326961,326961,becquèteront,['Troisième personne du pluriel du futur du ve...
73184,73184,compterait,['Du verbe compter.']
293721,293721,tabisèrent,['Troisième personne du pluriel du passé simpl...


In [137]:
tqdm.pandas(desc="Converting Definitions")
raw['Définitions'] = raw['Définitions'].progress_apply(ast.literal_eval) # safer than eval

# Explode the 'Définitions' list so each definition gets its own row

expanded_raw = raw.explode('Définitions')

C:\Users\Matthew\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Converting Definitions: 100%|███████████████████████████████████████████████| 973671/973671 [00:12<00:00, 77224.41it/s]


In [139]:
expanded_raw = expanded_raw.sort_values('Mot').reset_index(drop = True)

In [140]:
expanded_raw.drop(["index"], axis = 1, inplace = True)

In [141]:
expanded_raw.reset_index(inplace = True)

In [142]:
expanded_raw.head()

,index,Mot,Définitions
0,0,&amp;,variante ortho de et.
1,1,&amp;c.,variante ortho de etc.
2,2,'alif,variante ortho de alif.
3,3,'tain,Forme familière de putain.
4,4,'tis,Masculin pluriel de 'ti.


In [143]:
processed = expanded_raw.rename(columns = {"index" : "ID", "Mot" : "Word", "Définitions" : "Definition"})

In [144]:
processed

,ID,Word,Definition
0,0,&amp;,variante ortho de et.
1,1,&amp;c.,variante ortho de etc.
2,2,'alif,variante ortho de alif.
3,3,'tain,Forme familière de putain.
4,4,'tis,Masculin pluriel de 'ti.
...,...,...,...
1255989,1255989,ḳaf,Variante orthographique de kaf.
1255990,1255990,ṣād,Nom de la quatorzième lettre de l'alphabet ara...
1255991,1255991,ṭāʾ,Nom de la seizième lettre de l'alphabet arabe ...
1255992,1255992,ẓāʾ,variation orthographique de 'zza.


In [145]:
save_dir = "../../../ProcessedData/Dictionaries/French/"
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [146]:
processed.to_csv(save_dir + "kartmann.csv", encoding='utf-16', index=False)